In this notebook, we will apply the information-matching method to find the optimal PMU placements that leads to full system observability.
The objective is translated to requiring non-singular FIM of the system.
In practice, we set the target FIM to be a diagonal matrix, where the diagonal element is set to be some small positive number.
By doing this, the eigenvalues of the optimal FIM are larger than this small number, and thus the FIM is non-singular.

Details of the calculation:
* Model: IEEE 14-bus system
* Candidate data: Voltage phasor at each bus
* Eigenvalue cutoff: 1e-5

In [ ]:
from pathlib import Path

import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

from information_matching.convex_optimization import ConvexOpt
from information_matching.utils import tol, eps

%matplotlib inline
plt.style.use("default")

In [ ]:
# Set directories
case = 14  # 14-bus case

WORK_DIR = Path().absolute()
FIM_DIR = WORK_DIR / "FIMs"

# Setup

In [ ]:
# Model information
nparams = 2 * case

In [ ]:
# Configurations
nconfigs = case
configs = np.arange(case) + 1
# Identifier of the configurations/data
config_ids = np.array([f"bus{ii}" for ii in configs])

In [ ]:
# Load configuration FIMs
fim_configs_tensor = np.empty((nconfigs, nparams, nparams))
for ii, bus in enumerate(configs):
    I = np.loadtxt(FIM_DIR / f"fim_bus{bus}.csv", delimiter=",")
    fim_configs_tensor[ii] = I
fim_configs_tensor[0].shape

In [ ]:
# Target FIM
lambda_tol = 1e-5  # Eigenvalue cutoff
fim_target = np.diag(np.ones(nparams)) * lambda_tol
print("Eigenvalue lower bound:", lambda_tol)

# Convex optimization

Additional note:

We will perform the optimization iteratively.
In each iteration, we scale the weights being optimize by the optimal weights from the previous iteration.
By doing so, the objective function in the convex optimization better mimics $\ell_0$-norm to enforce sparsity.

In [ ]:
# Convex optimization
# Settings
cvx_tol = eps ** 0.75
solver = dict(verbose=False, solver=cp.SDPA, epsilonStar=cvx_tol)
print("Tolerance:", cvx_tol)

plt.figure()
wopt = np.ones(nconfigs)  # Initial weights
for ii in range(10):
    cvxopt = ConvexOpt(
        fim_target,
        fim_configs_tensor,
        np.array(config_ids),
        norm={"weights": wopt + 1e-15},  # Add some small number for numerical stability
        l1norm_obj=True,
    )

    try:
        # Solve
        cvxopt.solve(solver=solver)
        result = cvxopt.result.copy()
        wopt = result["wm"]
        dual = result["dual_wm"]
        print("Violation:", cvxopt.constraints[1].violation())
    except Exception:
        cvxopt.result = result
        break

    plt.plot(wopt, label=ii)

plt.yscale("log")
plt.xticks(range(nconfigs), config_ids, rotation=90)
plt.ylabel("Weights")
plt.legend(title="Iteration", bbox_to_anchor=(1, 1))
plt.show()

## Post-processing

In [ ]:
# Get the optimal buses
idx_wopt = cvxopt._get_idx_nonzero_wm(np.sqrt(cvx_tol))  # Index to the optimal buses
print("Optimal buses:")
_ = [print(f'{config_ids[ii]} \t {wopt[ii]}') for ii in idx_wopt]

Next, let's investigate whether these 3 buses are really sufficient.
To do this, we will compute the FIM of the configurations as we add more and more configurations, starting from the most optimal ones.
Then, we will compare the smallest eigenvalue of this FIM with the cutoff we set in the target FIM. 

In [ ]:
# FIM for each configuration, multiplied by their optimal weights
weighted_fims_configs = fim_configs_tensor * wopt.reshape((-1, 1, 1))

idx_sort = np.argsort(wopt)[::-1]
print("Buses sorted by weight magnitude: \n", configs[idx_sort])

for ii in np.arange(nconfigs) + 1:
    idx = idx_sort[:ii]
    I = np.sum(weighted_fims_configs[idx], axis=0)
    plt.figure()
    plt.title(
        f"Using the first {ii} optimal buses \n "
        + f"Smallest eigenvalue: {np.min(np.linalg.eigvalsh(I)):0.3e}"
    )
    cbound = np.max([-np.min(I), np.max(I)])
    plt.imshow(I, vmin=-cbound, vmax=cbound, cmap="bwr")
    plt.colorbar()
plt.show()

Notice that without these 3 optimal configurations, the smallest eigenvalue of the FIM is still lower than the cutoff.
However, as we have more than 3 configurations, the smallest eigenvalue doesn't really increase.

Note: Even though the smallest eigenvalue listed here when we have 3 or more configurations are smaller than the cutoff, but they are pretty close.
We believe this is just numerical artifact.